## 1 e 2. Importações, Path e Reprodutividade
Nesta primeira célula, resolvemos o problema do caminho das pastas e fixamos o seed para que seus resultados sejam consistentes.

In [1]:
import sys
import os
import numpy as np
import random as python_random
import polars as pl
import joblib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

# Ajuste de caminho para enxergar a pasta /src
sys.path.append(os.path.abspath(os.path.join('..')))
from src.database import execute_query
from src.const import QUERY_TREINAMENTO
from src.processing import *

# Reprodutividade
seed = 41
np.random.seed(seed)
python_random.seed(seed)
tf.random.set_seed(seed)

print("✅ Ambiente configurado e sementes plantadas!")

✅ Ambiente configurado e sementes plantadas!


## 3, 4 e 5. Carga, Conversão e Nulos
Aqui validamos se o banco entregou o que pedimos e se a limpeza inicial funcionou.

In [2]:
# 3. Ler dados brutos
df_raw = execute_query(QUERY_TREINAMENTO)

# 4 e 5. Conversão e Nulos (via pipe)
df_processado = (
    df_raw.with_columns([
        limpar_moeda(pl.col("valorsolicitado")),
        limpar_moeda(pl.col("valortotalbem"))
    ])
    .pipe(substituir_nulos)
)

print(f"Formato inicial: {df_raw.shape}")
print(f"Nulos após tratamento: {df_processado.null_count().sum().sum()}")

c:\temp\Udemy_Bootcamp_IA\src\database.py:25: UserWarning: Extension type 'arrow.opaque' is not registered; loading as its storage type.

To avoid this warning, register the extension type or set environment variable 'POLARS_UNKNOWN_EXTENSION_TYPE_BEHAVIOR' to 'load_as_storage' or 'load_as_extension'.

In Polars 2.0, the default behavior will change to 'load_as_extension'.
  return pl.read_database_uri(query=query, uri=uri, engine="adbc")


Formato inicial: (150, 13)
Nulos após tratamento: shape: (1, 13)
┌───────────┬─────────────┬───────┬─────────────┬───┬─────────┬─────────────┬─────────────┬────────┐
│ profissao ┆ tempoprofis ┆ renda ┆ tiporesiden ┆ … ┆ produto ┆ valorsolici ┆ valortotalb ┆ classe │
│ ---       ┆ sao         ┆ ---   ┆ cia         ┆   ┆ ---     ┆ tado        ┆ em          ┆ ---    │
│ u32       ┆ ---         ┆ u32   ┆ ---         ┆   ┆ u32     ┆ ---         ┆ ---         ┆ u32    │
│           ┆ u32         ┆       ┆ u32         ┆   ┆         ┆ u32         ┆ u32         ┆        │
╞═══════════╪═════════════╪═══════╪═════════════╪═══╪═════════╪═════════════╪═════════════╪════════╡
│ 0         ┆ 0           ┆ 0     ┆ 0           ┆ … ┆ 0       ┆ 0           ┆ 0           ┆ 0      │
└───────────┴─────────────┴───────┴─────────────┴───┴─────────┴─────────────┴─────────────┴────────┘


## 6, 7 e 8. Limpeza Avançada e Feature Engineering
Momento de validar a inteligência do FuzzyWuzzy e a nova coluna de proporção.

In [3]:
profissoes_validas = ['Advogado', 'Arquiteto', 'Cientista de Dados', 'Contador','Dentista','Empresário','Engenheiro','Médico','Programador']

df_final = (
    df_processado.pipe(corrigir_erros_digitacao, 'profissao', profissoes_validas)
    .pipe(tratar_outliers, 'tempoprofissao', 0, 70)
    .pipe(tratar_outliers, 'idade', 0, 110)
    .pipe(feature_engineering)
)

# Debug visual da Feature Engineering
display(df_final.select(['valorsolicitado', 'valortotalbem', 'proporcaosolicitadototal']).head())

valorsolicitado,valortotalbem,proporcaosolicitadototal
f64,f64,f64
8.4623e6,3.5e7,0.24178
1.26855e7,5e7,0.25371
1.27151e7,3.2e7,0.397347
2.8767e6,2.5e7,0.115068
1.99564e7,4e7,0.49891


## 9, 10 e 11. Divisão, Normalização e Codificação
Nesta etapa, saímos do Polars para o formato que a IA entende (Numpy/Pandas).

In [4]:
# 9. Divisão
X = df_final.drop('classe').to_pandas()
y = df_final.select('classe').to_pandas()['classe']

# Mapeamento da classe
mapeamento = {'ruim': 0, 'bom': 1}
y = y.map(mapeamento)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# 10 e 11. Normalização e Codificação (Salvando artefatos)
col_numericas = ['tempoprofissao','renda','idade','dependentes','valorsolicitado','valortotalbem','proporcaosolicitadototal']
col_categoricas = ['profissao', 'tiporesidencia', 'escolaridade','score','estadocivil','produto']

X_train = save_scalers(pl.from_pandas(X_train), col_numericas).to_pandas()
X_test = save_scalers(pl.from_pandas(X_test), col_numericas).to_pandas()

X_train = save_encoders(pl.from_pandas(X_train), col_categoricas).to_pandas()
X_test = save_encoders(pl.from_pandas(X_test), col_categoricas).to_pandas()

print("📈 Dados normalizados e categorizados com sucesso!")

📈 Dados normalizados e categorizados com sucesso!


## 12. Seleção de Atributos (RFE)
O toque final para descobrir o que realmente importa para o modelo.

In [5]:
print("🎯 Executando RFE (isso pode levar alguns segundos)...")
model = RandomForestClassifier(random_state=seed)
selector = RFE(model, n_features_to_select=10, step=1)
selector = selector.fit(X_train, y_train)

# Verificando quais colunas sobreviveram
colunas_eleitas = X_train.columns[selector.support_]
print(f"Colunas Selecionadas: {list(colunas_eleitas)}")

# Salvar seletor
joblib.dump(selector, '../objects/selector.joblib')

🎯 Executando RFE (isso pode levar alguns segundos)...
Colunas Selecionadas: ['profissao', 'tempoprofissao', 'renda', 'tiporesidencia', 'score', 'idade', 'estadocivil', 'valorsolicitado', 'valortotalbem', 'proporcaosolicitadototal']


['../objects/selector.joblib']